In [10]:
import pandas as pd
import plotly.express as px
import numpy as np

In [11]:
df = pd.read_csv('../experiments/data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df['time_from_start'] = df.groupby('experiment')['datetime'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600)


In [31]:

# Round both datetime and time_from_start
df['datetime'] = pd.to_datetime(df['datetime']).dt.round('s')
df['time_from_start'] = df['time_from_start'].round(2)  # Round to 2 decimal places

# Pivot using both rounded values
pivot_df = df.pivot_table(
    index=['datetime', 'time_from_start', 'experiment'], 
    columns='tag', 
    values='value',
    aggfunc='first'
).reset_index()

# Flatten column names
pivot_df.columns.name = None

In [33]:
# metric options
df['tag'].unique()

array(['Loss/train', 'Loss/val', 'Accuracy/train', 'Accuracy/val',
       'BLEU/val', 'LR'], dtype=object)

In [34]:
fig = px.line(pivot_df, 
              x='time_from_start', 
              y='Accuracy/val',
              color='experiment',
              markers=True,
              title='Loss Over Time')

fig.show()

### Creating Table

In [35]:
import os
import json

def load_experiment_args(dir):
    args_file = os.path.join(dir, 'args.json')
    if os.path.exists(args_file):
        with open(args_file, 'r') as f:
            return json.load(f)
    return None


In [36]:

# loop through each subdirectory in the experiments directory
experiments_dir = '../experiments'
experiment_args = {}
for subdir in os.listdir(experiments_dir):
    subdir_path = os.path.join(experiments_dir, subdir)
    if os.path.isdir(subdir_path):
        args = load_experiment_args(subdir_path)
        if args:
            experiment_args[subdir] = args

In [37]:
args_df = pd.DataFrame.from_dict(experiment_args, orient='index')
args_df

,data_dir,output_dir,encoder_d,decoder_d,embed_d,attention_d,num_layers,dropout,pretrained,epochs,...,lr,weight_decay,grad_clip,accumulation_steps,patience,num_workers,resume,seed,num_heads,dim_feedforward
exp1,crohme_images,experiments/exp1,256,256,256,256,2,0.50,False,100,...,0.0010,0.00010,5.0,1,10,4,None,42,NaN,NaN
exp2,crohme_images,experiments/exp2,64,64,64,64,1,0.50,False,200,...,0.0050,0.00010,5.0,1,10,8,None,42,NaN,NaN
exp3,crohme_images,experiments/exp3,512,512,512,512,3,0.45,True,200,...,0.0005,0.00005,1.0,1,20,6,None,42,NaN,NaN
sanity_check,data/crohme_images,experiments/sanity_check,256,256,256,256,2,0.50,False,1,...,0.0010,0.00010,5.0,1,10,4,None,42,NaN,NaN
transformer_run,data/crohme_images,experiments/transformer_run,256,256,256,256,3,0.10,True,5,...,0.0010,0.00010,5.0,1,10,4,None,42,8.0,1024.0
